In [59]:
# import libraries
from sodapy import Socrata
import pandas as pd
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from datetime import datetime, timedelta, date
import os
import requests
import time
import pyarrow

### WIP:   
### Select only columns in API request without losing data




### Get full year of COVID-19 nursing home data with CMS open API. The variable, latest_uuid will need to be updated according to uuid version updates per https://data.cms.gov/covid-19/covid-19-nursing-home-data/api-docs

In [14]:
# API latest_uuid
latest_uuid = '	70891227-7659-4472-abe7-92d03f264fc8'


In [15]:
url1 = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]=2021-06-27&offset=0&size=5000"
response1 = requests.get(url1)
response1


<Response [200]>

In [16]:
# gets all data for a single week up to offset of 15500

nh_results_m1 = []
size=5000

for i in range(0,15500,5000):
    print("----")
    offset = i
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]=2021-06-27&offset={offset}&size={size}"
    print("requesting", url)

    response = requests.get(url)
    nh_data = response.json()
    nh_results_m1.extend(nh_data)
  

----
requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=0&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=5000&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=10000&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=15000&size=5000


In [17]:
len(nh_results_m1)

15256

In [18]:
nh_results_m1[-1]

{'week_ending': '2021-06-27',
 'federal_provider_number': '686124',
 'provider_name': 'HARMONY HEALTH CENTER',
 'provider_address': '9820 N KENDALL DRIVE',
 'provider_city': 'MIAMI',
 'provider_state': 'FL',
 'provider_zip_code': '33176',
 'provider_phone_number': '3052716311',
 'county': 'Miami-Dade',
 'submitted_data': 'Y',
 'passed_quality_assurance_check': 'Y',
 'residents_weekly_admissions_covid_19': '0',
 'residents_total_admissions_covid_19': '23',
 'residents_weekly_confirmed_covid_19': '0',
 'residents_total_confirmed_covid_19': '39',
 'residents_weekly_all_deaths': '1',
 'residents_total_all_deaths': '46',
 'residents_weekly_covid_19_deaths': '0',
 'residents_total_covid_19_deaths': '2',
 'number_of_all_beds': '203',
 'total_number_of_occupied_beds': '192',
 'staff_weekly_confirmed_covid_19': '0',
 'staff_total_confirmed_covid_19': '61',
 'staff_weekly_covid_19_deaths': '0',
 'staff_total_covid_19_deaths': '0',
 'shortage_of_nursing_staff': 'N',
 'shortage_of_clinical_staff':

### Each week returns approx 15250 rows, sometimes more, but does not exceed 15500. I have set up a nested loop (whole/for) to accomodate offset and date increments. It retrieves eight weeks of data, which is saved in a dataframe. For 8 wks, num of rows = approx 122000.
### To get a year's worth of data, I re-run this process for each 8(ish) week chunk between 6-27-21 and 7-10-22.

### Each 8-wk dataframe is then subset to keep only needed cols before concatenation. 

### Excluding rows for which "passed data quality assurance check" = 'N'

In [19]:
# First batch: week ending 6-27-21 to week ending 8-15-21

nh_all_Jun27_21_Aug15_21 = []
week_ending_start = date(2021,6,27)
week_ending_end = date(2021,8,15)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Jun27_21_Aug15_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-06-27&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-07-04&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [20]:
nh_all_Jun27_21_Aug15_21 = pd.DataFrame(nh_all_Jun27_21_Aug15_21)
nh_all_Jun27_21_Aug15_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,62.069,1.149,63.218,98.182,,,0,,,0
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,33.645,0,33.645,100,,,0,,,0
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,62.857,7.143,70,,,,0,,,0
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,68.132,0,68.132,,,,0,,,0
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,66.038,0.943,66.981,,,,0,,,0


In [21]:
nh_all_Jun27_21_Aug15_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121988,2021-08-15,676499,SPRINGTOWN PARK REHABILITATION AND CARE CENTER,201 WILLIAMS WARD RD.,SPRINGTOWN,TX,76082,5124382630,Parker,Y,...,81.034,8.621,89.655,83.929,,,0,,,0
121989,2021-08-15,686123,KENDALL LAKES HEALTH AND REHABILITATION CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,...,78.226,0,78.226,,,,0,,,0
121990,2021-08-15,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,64.593,1.914,66.507,97.122,,,0,,,0
121991,2021-08-15,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,N,...,,,,,,,,,,
121992,2021-08-15,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,92.308,5.128,97.436,94.737,,,0,,,0


In [22]:
nh_all_Jun27_21_Aug15_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121993 entries, 0 to 121992
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 94.0+ MB


In [23]:
# Second batch: 8-22-21 to 10-10-21

nh_all_Aug22_21_Oct10_21 = []
week_ending_start = date(2021,8,22)
week_ending_end = date(2021,10,10)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Aug22_21_Oct10_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-22&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-22&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-22&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-22&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-22&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-08-29&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [24]:
nh_all_Aug22_21_Oct10_21 = pd.DataFrame(nh_all_Aug22_21_Oct10_21)
nh_all_Aug22_21_Oct10_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2021-08-22,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,63.218,0,63.218,100,,,0,,,0
1,2021-08-22,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,42.708,3.125,45.833,93.182,,,0,,,0
2,2021-08-22,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,71.429,5.714,77.143,,,,0,,,0
3,2021-08-22,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,79.012,1.235,80.247,,,,0,,,0
4,2021-08-22,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,65.714,0.952,66.667,,,,0,,,0


In [25]:
nh_all_Aug22_21_Oct10_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121942,2021-10-10,686123,KENDALL LAKES HEALTH AND REHABILITATION CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,...,84.388,0,84.388,100,85,,67.46,62,,31
121943,2021-10-10,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,80.5,0,80.5,100,0,,0,0,,0
121944,2021-10-10,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,0,0,,0,0,,0,0,,0
121945,2021-10-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,93.75,2.083,95.833,97.826,0,,0,1,,2.222
121946,2021-10-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,60,0,60,100,0,,0,0,,0


In [26]:
nh_all_Aug22_21_Oct10_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121947 entries, 0 to 121946
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 94.0+ MB


In [27]:
# Third batch: 10-17-21 to 12-05-21

nh_all_Oct17_21_Dec05_21 = []
week_ending_start = date(2021,10,17)
week_ending_end = date(2021,12,5)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Oct17_21_Dec05_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-17&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-17&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-17&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-17&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-17&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-10-24&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [28]:
nh_all_Oct17_21_Dec05_21 = pd.DataFrame(nh_all_Oct17_21_Dec05_21)
nh_all_Oct17_21_Dec05_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2021-10-17,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,70.115,3.448,73.563,95.313,0,,0,0,,0
1,2021-10-17,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,46.465,0,46.465,100,0,,0,2,,4.348
2,2021-10-17,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,78.205,0,78.205,100,23,,62.162,16,,26.23
3,2021-10-17,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,74.444,7.778,82.222,90.541,54,,84.375,29,,43.284
4,2021-10-17,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,71.134,1.031,72.165,98.571,0,,0,0,,0


In [29]:
nh_all_Oct17_21_Dec05_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121851,2021-12-05,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,82.703,5.405,88.108,93.865,91,,77.119,36,,23.529
121852,2021-12-05,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,86.765,7.353,94.118,90.769,12,,30.769,4,,6.78
121853,2021-12-05,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,85.965,7.018,92.982,92.453,24,,82.759,13,,26.531
121854,2021-12-05,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,62.5,0,62.5,100,0,,0,1,,20
121855,2021-12-05,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [30]:
nh_all_Oct17_21_Dec05_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121856 entries, 0 to 121855
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 93.9+ MB


In [31]:
# Fourth batch: 12-12-21 to 1-30-22

nh_all_Dec12_21_Jan30_22 = []
week_ending_start = date(2021,12,12)
week_ending_end = date(2022,1,30)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Dec12_21_Jan30_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-12&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-12&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-12&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-12&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-12&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2021-12-19&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [32]:
nh_all_Dec12_21_Jan30_22 = pd.DataFrame(nh_all_Dec12_21_Jan30_22)
nh_all_Dec12_21_Jan30_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2021-12-12,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,69.767,12.791,82.558,84.507,38,,80.851,34,,56.667
1,2021-12-12,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,54.737,23.158,77.895,70.27,64,,88.889,4,,7.692
2,2021-12-12,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,84.507,9.859,94.366,89.552,35,,92.105,27,,45
3,2021-12-12,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,92.784,5.155,97.938,94.737,59,,83.099,58,,64.444
4,2021-12-12,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,75.258,3.093,78.351,96.053,46,,67.647,37,,50.685


In [33]:
nh_all_Dec12_21_Jan30_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121699,2022-01-30,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,82.716,6.173,88.889,85.897,34,,66.667,10,,14.925
121700,2022-01-30,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,89.063,4.688,93.75,93.443,23,,88.462,33,,57.895
121701,2022-01-30,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
121702,2022-01-30,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,60,20,80,75,0,,0,2,,33.333
121703,2022-01-30,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [34]:
nh_all_Dec12_21_Jan30_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121704 entries, 0 to 121703
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 93.8+ MB


In [36]:
# Fifth batch: 2-6-22 to 3-27-22

nh_all_Feb06_22_Mar27_22 = []
week_ending_start = date(2022,2,6)
week_ending_end = date(2022,3,27)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Feb06_22_Mar27_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-06&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-06&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-06&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-06&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-06&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-02-13&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [37]:
nh_all_Feb06_22_Mar27_22 = pd.DataFrame(nh_all_Feb06_22_Mar27_22)
nh_all_Feb06_22_Mar27_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2022-02-06,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,82.353,7.059,89.412,92.105,35,,72.917,28,,40
1,2022-02-06,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,70.526,3.158,73.684,95.714,64,,94.118,4,,5.97
2,2022-02-06,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,92.683,0,92.683,96.203,36,,97.297,40,,52.632
3,2022-02-06,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,94.845,2.062,96.907,97.872,59,,84.286,46,,50
4,2022-02-06,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,79.31,9.483,88.793,89.32,58,,84.058,44,,47.826


In [38]:
nh_all_Feb06_22_Mar27_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121571,2022-03-27,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,95.833,0,95.833,100,3,,10.714,55,,79.71
121572,2022-03-27,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
121573,2022-03-27,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,80,0,80,100,0,,0,2,,50
121574,2022-03-27,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,...,,,,,,,,,,
121575,2022-03-27,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [39]:
nh_all_Feb06_22_Mar27_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121576 entries, 0 to 121575
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 93.7+ MB


In [40]:
# Sixth batch: 4-3-22 to 5-22-22

nh_all_Apr03_22_May22_22 = []
week_ending_start = date(2022,4,3)
week_ending_end = date(2022,5,22)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Apr03_22_May22_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-03&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-03&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-03&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-03&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-03&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-04-10&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [41]:
nh_all_Apr03_22_May22_22=pd.DataFrame(nh_all_Apr03_22_May22_22)
nh_all_Apr03_22_May22_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2022-04-03,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,87.059,2.353,89.412,97.368,36,,78.261,28,,37.838
1,2022-04-03,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,76.923,1.648,78.571,97.902,63,,91.304,16,,11.429
2,2022-04-03,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,96.154,0,96.154,100,36,,97.297,50,,66.667
3,2022-04-03,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,92.553,3.191,95.745,96.667,57,,82.609,55,,63.218
4,2022-04-03,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,81.871,6.433,88.304,92.715,55,,75.342,62,,44.286


In [42]:
nh_all_Apr03_22_May22_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
121488,2022-05-22,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,94.667,0,94.667,100,19,,63.333,31,,43.662
121489,2022-05-22,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
121490,2022-05-22,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,83.333,0,83.333,100,0,,0,1,,10
121491,2022-05-22,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,Y,...,97.101,0,97.101,100,4,,44.444,54,,80.597
121492,2022-05-22,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [43]:
nh_all_Apr03_22_May22_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121493 entries, 0 to 121492
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 93.6+ MB


In [44]:
# Seventh batch: 5-29-22 to 7-10-22

nh_all_May29_22_Jul10_22 = []
week_ending_start = date(2022,5,29)
week_ending_end = date(2022,7,10)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_May29_22_Jul10_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(5)

---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-05-29&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-05-29&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-05-29&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-05-29&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-05-29&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data?filter[week_ending]=2022-06-05&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/5dbd3871-abbb-4131-a248-0e57a402b245/data

In [46]:
nh_all_May29_22_Jul10_22=pd.DataFrame(nh_all_May29_22_Jul10_22)
nh_all_May29_22_Jul10_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
0,2022-05-29,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,88,0,88,100,49,,98,28,,42.424
1,2022-05-29,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,77.778,0,77.778,100,63,,92.647,17,,11.565
2,2022-05-29,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,96.203,0,96.203,100,36,,100,54,,71.053
3,2022-05-29,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,N,...,,,,,,,,,,
4,2022-05-29,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,85.455,2.727,88.182,96.907,65,,89.041,60,,63.83


In [47]:
nh_all_May29_22_Jul10_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_or_partial_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_excluding_declinations_from_eligible_personnel_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,number_of_healthcare_personnel_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
106238,2022-07-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,95.181,0,95.181,100,25,83.333,83.333,39,49.367,49.367
106239,2022-07-10,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
106240,2022-07-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,83.333,0,83.333,100,0,0,0,1,10,10
106241,2022-07-10,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,...,,,,,,44.444,,,83.582,
106242,2022-07-10,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [48]:
nh_all_May29_22_Jul10_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106243 entries, 0 to 106242
Columns: 101 entries, week_ending to percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time
dtypes: object(101)
memory usage: 81.9+ MB


## Subset data

In [49]:
# from each df, keep only needed cols

needed_cols = [
                'week_ending','federal_provider_number','provider_name','provider_address',
                'provider_city','provider_state','provider_zip_code','provider_phone_number',
                'county','submitted_data','passed_quality_assurance_check','residents_weekly_confirmed_covid_19',
                'residents_weekly_all_deaths','residents_weekly_covid_19_deaths','number_of_all_beds','total_number_of_occupied_beds',
                'staff_weekly_confirmed_covid_19','shortage_of_nursing_staff','shortage_of_clinical_staff','shortage_of_aides',
                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents','number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week','number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time','percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week','number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time','percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time'
]

nh_sub_Jun27_21_Aug15_21 = nh_all_Jun27_21_Aug15_21[needed_cols]

nh_sub_Aug22_21_Oct10_21 = nh_all_Aug22_21_Oct10_21[needed_cols]

nh_sub_Oct17_21_Dec05_21 = nh_all_Oct17_21_Dec05_21[needed_cols]

nh_sub_Dec12_21_Jan30_22 = nh_all_Dec12_21_Jan30_22[needed_cols]

nh_sub_Feb06_22_Mar27_22 = nh_all_Feb06_22_Mar27_22[needed_cols]

nh_sub_Apr03_22_May22_22 = nh_all_Apr03_22_May22_22[needed_cols]

nh_sub_May29_22_Jul10_22 = nh_all_May29_22_Jul10_22[needed_cols]

In [50]:
nh_sub_Jun27_21_Aug15_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,79,67,,84.81,84.81,106,70,,66.038,66.038


In [51]:
nh_sub_Aug22_21_Oct10_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-08-22,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,54,51,2,94.444,96.226,87,55,32,63.218,63.218
1,2021-08-22,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,80,74,6,92.5,92.5,96,41,52,42.708,42.708
2,2021-08-22,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,41,37,,90.244,90.244,70,50,,71.429,71.429
3,2021-08-22,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,70,64,,91.429,91.429,81,64,,79.012,79.012
4,2021-08-22,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,78,69,,88.462,88.462,105,69,,65.714,65.714


In [52]:
nh_sub_Aug22_21_Oct10_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
121942,2021-10-10,686123,KENDALL LAKES HEALTH AND REHABILITATION CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,...,147,126,21,85.714,85.714,237,200,37,84.388,84.388
121943,2021-10-10,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,178,129,49,72.472,72.472,200,161,39,80.5,80.5
121944,2021-10-10,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,0,0,0,0,0,0,0,0,0,0
121945,2021-10-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,19,18,1,94.737,94.737,48,45,2,93.75,93.75
121946,2021-10-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,1,1,0,100,100,5,3,2,60,60


In [55]:
#concat dfs
nh_sub_Jun27_21_Jul10_22_concat = pd.concat([nh_sub_Jun27_21_Aug15_21, nh_sub_Aug22_21_Oct10_21, nh_sub_Oct17_21_Dec05_21, nh_sub_Dec12_21_Jan30_22, nh_sub_Feb06_22_Mar27_22, nh_sub_Apr03_22_May22_22, nh_sub_May29_22_Jul10_22]).reset_index(drop=True)

In [56]:
nh_sub_Jun27_21_Jul10_22_concat.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,54,43,2,79.63,81.132,87,54,32,62.069,62.069
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,79,65,13,82.278,82.278,107,36,71,33.645,33.645
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,44,41,2,93.182,93.182,70,44,,62.857,62.857
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,68,64,,94.118,94.118,91,62,,68.132,68.132
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,79,67,,84.81,84.81,106,70,,66.038,66.038


In [57]:
nh_sub_Jun27_21_Jul10_22_concat.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time
836807,2022-07-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,31,30,1,96.774,96.774,83,79,4,95.181,95.181
836808,2022-07-10,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,N,...,,,,,,,,,,
836809,2022-07-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,1,0,1,0,0,12,10,2,83.333,83.333
836810,2022-07-10,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,...,,,,,,,,,,
836811,2022-07-10,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,N,...,,,,,,,,,,


In [58]:
nh_sub_Jun27_21_Jul10_22_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836812 entries, 0 to 836811
Data columns (total 32 columns):
 #   Column                                                                                                                                                          Non-Null Count   Dtype 
---  ------                                                                                                                                                          --------------   ----- 
 0   week_ending                                                                                                                                                     836812 non-null  object
 1   federal_provider_number                                                                                                                                         836812 non-null  object
 2   provider_name                                                                                                                   

## Save df to parquet

In [61]:
nh_sub_Jun27_21_Jul10_22_concat.to_parquet('data/nh_sub_Jun27_21_Jul10_22_concat.parquet.gzip',
                                          compression='gzip')

#### Thanks to: Jonathan Soma's API tutorial: https://www.youtube.com/watch?v=4Fdyft-ky0w and Jennifer Peebles' documentation on working with the CMS COVID-19 Nursing Home data API: https://github.com/jenniferpeebles/covid_in_nursing_homes_data_from_cms
